In [1]:
import sys
sys.path.append('..')
import flickrapi
import nationalparks.secrets as secrets

In [31]:
## start API
flickr = flickrapi.FlickrAPI(secrets.api_key, secrets.api_secret, format='parsed-json')

In [ ]:
## what do we want it to do

## collect park images
## populate database
## filter images based on locations
## connect to park database
## 


class FlickrParkFinder():
    def __init__(self, park):
        ## set API connection
        self.flickr = flickrapi.FlickrAPI(secrets.api_key, secrets.api_secret, format='parsed-json')
        self.park = park
    
    def 

In [45]:
class FlickrImage():
    '''
    Flickr Image class
    Constructor:
        inputs:
            data from flickr.photos.search
    Methods:
        get_photo_url: return formatted url
        get_info: fetch detailed information about the image
    '''
    def __init__(self, data):
        self.id = data['id']
        self.owner = data['owner']
        self.secret = data['secret']
        self.server = data['server']
        self.farm = data['farm']
        self.title = data['title']
        self.owner = data['owner']
        self.get_info()
        
    def get_photo_url(self):
        ## format url to image
        return "https://farm{0}.staticflickr.com/{1}/{2}_{3}.jpg".format(self.farm, self.server, self.id, self.secret)
        
    def get_info(self):
        ## fetch info
        result = flickr.photos.getInfo(photo_id=self.id)
        ## parse info
        self.owner = result['photo']['owner']['username']
        self.dateposted = result['photo']['dates']['taken']
        self.tags = [x['raw'] for x in result['photo']['tags']['tag']]
        self.lon = result['photo']['location']['longitude']
        self.lat = result['photo']['location']['latitude']
        self.accuracy = result['photo']['location']['accuracy']
        self.context = result['photo']['location']['context']
        self.locality = result['photo']['location']['locality']['_content']
        self.county = result['photo']['location']['county']['_content']
        self.region = result['photo']['location']['region']['_content']
        self.country = result['photo']['location']['country']['_content']
        self.neighbourhood = result['photo']['location']['neighbourhood']['_content']
        self.url = result['photo']['urls']['url'][0]['_content']

In [ ]:
https://www.flickr.com/photo.gne?id=49847312746

In [41]:
## get photo info
results = flickr.photos.getInfo(photo_id="49939991706")
results

{'photo': {'id': '49939991706',
  'secret': '0efec429af',
  'server': '65535',
  'farm': 66,
  'dateuploaded': '1590539494',
  'isfavorite': 0,
  'license': '0',
  'safety_level': '0',
  'rotation': 0,
  'originalsecret': '946fbaecff',
  'originalformat': 'jpg',
  'owner': {'nsid': '94881114@N04',
   'username': 'The Flying Inn',
   'realname': 'The Flying Inn',
   'location': 'Newark, Delaware, USA',
   'iconserver': '5469',
   'iconfarm': 6,
   'path_alias': None},
  'title': {'_content': 'Pemetic Trail'},
  'description': {'_content': "Taken last fall. Going through some old photos and finding things I didn't think much of at the time that have grown on me."},
  'visibility': {'ispublic': 1, 'isfriend': 0, 'isfamily': 0},
  'dates': {'posted': '1590539494',
   'taken': '2019-06-14 00:59:40',
   'takengranularity': '0',
   'takenunknown': '0',
   'lastupdate': '1590543039'},
  'views': '80',
  'editability': {'cancomment': 0, 'canaddmeta': 0},
  'publiceditability': {'cancomment': 1,

In [21]:
## get location of specific image
results = flickr.photos.geo.getLocation(photo_id="49939991706")
results

{'photo': {'id': '49939991706',
  'location': {'latitude': '44.343817',
   'longitude': '-68.245739',
   'accuracy': '14',
   'context': '0',
   'locality': {'_content': 'Mount Desert'},
   'county': {'_content': 'Hancock'},
   'region': {'_content': 'Maine'},
   'country': {'_content': 'United States'},
   'neighbourhood': {'_content': 'Otter Creek'}}},
 'stat': 'ok'}

In [35]:
image = FlickrImage(sets['photos']['photo'][0])

In [37]:
image.get_tags()
image.tags

['Acadia',
 'Maine',
 'National Park',
 'Pemetic Mountsin',
 'Trail',
 'nature',
 'rocks',
 'stream',
 'trees',
 'waterfall',
 'woods']

In [42]:
## search
sets= flickr.photos.search(text='Acadia National Park', privacy_filter=1, has_geo=True, bbox="-68.25,-68.20,44.3,44.35")
sets

{'photos': {'page': 1,
  'pages': 25,
  'perpage': 250,
  'total': '6217',
  'photo': [{'id': '49884665796',
    'owner': '51771794@N07',
    'secret': '9620eae7c4',
    'server': '65535',
    'farm': 66,
    'title': 'Maine Breaking Waves',
    'ispublic': 1,
    'isfriend': 0,
    'isfamily': 0},
   {'id': '49881539723',
    'owner': '37317281@N08',
    'secret': '81c5dc825f',
    'server': '65535',
    'farm': 66,
    'title': 'Near Sand Beach, Acadia National Park, ME, September, 2014',
    'ispublic': 1,
    'isfriend': 0,
    'isfamily': 0},
   {'id': '49847850786',
    'owner': '51546668@N00',
    'secret': '26c9be4bc7',
    'server': '65535',
    'farm': 66,
    'title': 'otter cove',
    'ispublic': 1,
    'isfriend': 0,
    'isfamily': 0},
   {'id': '49847312746',
    'owner': '51546668@N00',
    'secret': 'f3f7ebd53d',
    'server': '65535',
    'farm': 66,
    'title': 'rocky shorelines',
    'ispublic': 1,
    'isfriend': 0,
    'isfamily': 0},
   {'id': '49847311601',
   